In [9]:
%load_ext autoreload
%autoreload 2
import argparse
import dataclasses
import datetime
import itertools
import logging
import pathlib
import os

import numpy as np
import pandas as pd
import sklearn.ensemble as ensemble
import sklearn.metrics
import sklearn.model_selection
import xarray as xr
import torch

import a6
import a6.datasets.coordinates as _coordinates
import a6.datasets.variables as _variables
import a6.utils as utils

WORKER_ID = 10

utils.logging.create_logger(
    global_rank=WORKER_ID,
    local_rank=WORKER_ID,
    verbose=True,
)

logger = logging.getLogger("notebook")

turbine_data_dir = pathlib.Path(
    "/p/home/jusers/emmerich1/juwels/data/production"
)
preprocessed_data_dir = pathlib.Path(
    "/p/home/jusers/emmerich1/juwels/data/production-processed"
)
results_dir = pathlib.Path(
    "/p/project/deepacf/maelstrom/emmerich1/data/forecast-errors"
)

turbine_files = a6.utils.paths.list_files(
    turbine_data_dir, pattern="**/*.nc", recursive=True
)

results = xr.open_dataset("/p/project/deepacf/maelstrom/emmerich1/data/pca_kpca_kmeans_lswrs_29_40.nc")
results_40 = results.sel(k=40)
gwl = xr.open_dataset("/p/home/jusers/emmerich1/juwels/code/a6/src/tests/data/gwl.nc")
dcv2 = xr.open_dataset("/p/project/deepacf/maelstrom/emmerich1/data/dcv2-lswrs.nc")

lswrs = [None, gwl["GWL"], results_40["PCA"], results_40["kPCA"], dcv2["DCv2"]]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
@dataclasses.dataclass
class Errors:
    nmae: float
    nrmse: float


def _calculate_nmae_and_nrmse(
    start: datetime.datetime,
    end: datetime.datetime,
    gs: sklearn.model_selection.GridSearchCV,
    weather_data: list[xr.DataArray],
    turbine: xr.Dataset,
    power_rating: float,
    turbine_variables: _variables.Turbine,
    coordinates: _coordinates.Coordinates,
) -> Errors:
    window = slice(start, end)
    logger.debug("Evaluating model error for slice=%s", window)

    weather_forecast = [d.sel({coordinates.time: window}) for d in weather_data]
    X_forecast = a6.features.methods.reshape.sklearn.transpose(  # noqa: N806
        *weather_forecast
    )

    turbine_sub = turbine.sel({coordinates.time: window})[
        turbine_variables.production
    ]
    y_true = a6.features.methods.reshape.sklearn.transpose(turbine_sub)

    if y_true.size == 0:
        logger.warning(
            (
                "Emtpy production data for start=%s "
                "end=%s, setting errors to NaN"
            ),
            start,
            end,
        )
        return Errors(np.nan, np.nan)

    y_pred = gs.predict(X_forecast)

    nmae = a6.training.metrics.turbine.calculate_nmae(
        y_true=y_true, y_pred=y_pred, power_rating=power_rating
    )
    nrmse = a6.training.metrics.turbine.calculate_nrmse(
        y_true=y_true, y_pred=y_pred, power_rating=power_rating
    )
    return Errors(nmae=nmae, nrmse=nrmse)


In [12]:

if WORKER_ID is not None and WORKER_ID >= len(turbine_files):
    logger.warning("Exiting: no file to process")
    raise RuntimeError()

coordinates: _coordinates.Coordinates = _coordinates.Coordinates()
turbine_variables: _variables.Turbine = a6.datasets.variables.Turbine()

result = {}

for i, turbine_path in enumerate(turbine_files):
    if WORKER_ID is not None and i != WORKER_ID:
        continue

    logger.info(
        "Processing turbine %i/%i (path=%s)",
        i,
        len(turbine_files),
        turbine_path,
    )

    turbine_name = turbine_path.name.replace(".nc", "")

    turbine_path: pathlib.Path = (
        preprocessed_data_dir / f"{turbine_name}/turbine.nc"
    )
    pl_path: pathlib.Path = (
        preprocessed_data_dir / f"{turbine_name}/pl.nc"
    )
    ml_path: pathlib.Path = (
        preprocessed_data_dir / f"{turbine_name}/ml.nc"
    )
    sfc_path: pathlib.Path = (
        preprocessed_data_dir / f"{turbine_name}/sfc.nc"
    )

    logger.info("Reading preprocessed data")
    turbine = xr.open_dataset(turbine_path)
    pl = xr.open_dataset(pl_path)
    ml = xr.open_dataset(ml_path)
    sfc = xr.open_dataset(sfc_path)

    power_rating = turbine_variables.read_power_rating(turbine)
    logger.info("Extracted power rating %i", power_rating)
    
    start, end = min(turbine["time"].values), max(turbine["time"].values)
    dates = pd.date_range(start, end, freq="1d")
    logger.info(
        "Simulating forecast errors for LSWRS %s for date range %s-%s",
        lswrs,
        start,
        end,
    )

    forecast_errors = {}
    
    for lswr in lswrs:
        lswr_name = "none" if lswr is None else lswr.name
        
        logger.info("Handling LSWR %s", lswr_name)
        
        outfile: pathlib.Path = (
            results_dir / f"{turbine_name}-forecast-errors-lswr-{lswr_name}.nc"
        )

        if outfile.exists():
            logger.warning(
                "Skipping %s since outfile already exists at %s",
                turbine_path,
                outfile,
            )
            
        data = [ml[var] for var in ml.data_vars] + [sfc[var] for var in sfc.data_vars] + [pl[var] for var in pl.data_vars]
        categorical_features = [False for _ in enumerate(data)]
        
        if lswr is not None:
            lswr_labels = lswr.sel(time=turbine[coordinates.time], method="nearest")
            data.append(lswr_labels)
            categorical_features.append(True)
        
        logger.info(
            "Preparing input data for variables %s", [d.name for d in data]
        )

        X = a6.features.methods.reshape.sklearn.transpose(*data)  # noqa: N806
        y = a6.features.methods.reshape.sklearn.transpose(
            turbine[turbine_variables.production]
        )

        (  # noqa: N806
            X_train,
            _,
            y_train,
            _,
        ) = sklearn.model_selection.train_test_split(X, y, train_size=1/3)

        logger.info(
            "Train dataset size is %i hours (~%i days)",
            y_train.size,
            y_train.size // 24,
        )

        logger.info("Fitting model with GridSearchCV")


        param_grid = {
            "learning_rate": [0.03, 0.05, 0.07, 0.1],
            "l2_regularization": [0.0, 1.0, 3.0, 5.0, 7.0],
            "max_iter": [200, 300, 500],
            "max_depth": [15, 37, 63, 81],
            "min_samples_leaf": [23, 48, 101, 199],
            "categorical_features": [categorical_features],
        }
        param_grid = {"learning_rate": [0.1]}
        n_jobs = int(a6.utils.get_cpu_count())

        gs = sklearn.model_selection.GridSearchCV(
            estimator=ensemble.HistGradientBoostingRegressor(
                loss="squared_error"
            ),
            param_grid=param_grid,
            scoring=sklearn.metrics.make_scorer(
                a6.training.metrics.turbine.calculate_nrmse,
                greater_is_better=False,
                power_rating=power_rating,
            ),
            # 10-fold CV
            cv=2,
            refit=True,
            n_jobs=n_jobs,
        )
        gs = gs.fit(X=X_train, y=y_train.ravel())

        results: list[Errors] = [
            _calculate_nmae_and_nrmse(
                start=start,
                end=end,
                gs=gs,
                weather_data=data,
                turbine=turbine,
                power_rating=power_rating,
                turbine_variables=turbine_variables,
                coordinates=coordinates,
            )
            for start, end in itertools.pairwise(dates[:3])
        ]
        forecast_errors[lswr_name] = results

RANK 10 (LOCAL 10) - INFO - 2024-03-28 18:13:42 - 0:00:34 - Processing turbine 10/45 (path=/p/home/jusers/emmerich1/juwels/data/production/hohw1.nc)
RANK 10 (LOCAL 10) - INFO - 2024-03-28 18:13:42 - 0:00:34 - Reading preprocessed data
RANK 10 (LOCAL 10) - INFO - 2024-03-28 18:13:42 - 0:00:34 - Extracted power rating 2000
RANK 10 (LOCAL 10) - INFO - 2024-03-28 18:13:42 - 0:00:35 - Simulating forecast errors for LSWRS [None, <xarray.DataArray 'GWL' (time: 15777)>
                                                            [15777 values with dtype=int64]
                                                            Coordinates:
                                                              * time     (time) datetime64[ns] 1979-07-01 1979-07-02 ... 2022-10-13, <xarray.DataArray 'PCA' (time: 21826)>
                                                            [21826 values with dtype=int32]
                                                            Coordinates:
                                

In [13]:
def unpack_errors_per_method(errors, attr: str):
    return [
        [getattr(error, attr) for error in method]
        for method in zip(*errors.values())
    ]

In [16]:
coords = {"time": dates[:2], "lswr_method": list(forecast_errors.keys())}
dims = ["time", "lswr_method"]
                                     
nmae_da = xr.DataArray(
    unpack_errors_per_method(forecast_errors, attr="nmae"),
    coords=coords,
    dims=dims,
)
nrmse_da = xr.DataArray(
    unpack_errors_per_method(forecast_errors, attr="nrmse"),
    coords=coords,
    dims=dims,
)
errors = xr.Dataset(
    data_vars={"nmae": nmae_da, "nrmse": nrmse_da},
    coords=nmae_da.coords,
)